# Fase 1: Importar las dependencias


In [3]:
import numpy as np
import math
import re                     #expresiones regulares
import pandas as pd
from bs4 import BeautifulSoup #para trabajar con txt en diferentes formatos

from google.colab import drive#esto es solamente en GoogleColab

In [4]:
try:
    %tensorflow_version 2.x
except Exception:
    pass
import tensorflow as tf
#esto es solamente en GoogleColab, es para cargar la última TF

from tensorflow.keras import layers
import tensorflow_datasets as tfds #de aquí tomaremos el 'tokenizador'

# Fase 2: Pre Procesado de Datos

## Carga de Ficheros

In [5]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
nombres_df = pd.read_csv('/content/drive/Othercomputers/Mi PC/02. Programacion/Proyectos/prediction-of-gender-names/name_gender_dataset.csv')

Data exploration

In [5]:
nombres_df.shape

(147269, 4)

In [6]:
nombres_df.head(5)

,Name,Gender,Count,Probability
0,James,M,5304407,0.014517
1,John,M,5260831,0.014398
2,Robert,M,4970386,0.013603
3,Michael,M,4579950,0.012534
4,William,M,4226608,0.011567


Data cleaning

In [7]:
nombres_df = nombres_df.drop(['Count'], axis=1)
nombres_df = nombres_df.drop(['Probability'], axis=1)

nombres_df.head(5)

,Name,Gender
0,James,M
1,John,M
2,Robert,M
3,Michael,M
4,William,M


In [8]:
nombres_df = nombres_df[['Gender','Name']]

nombres_df.head(5)

,Gender,Name
0,M,James
1,M,John
2,M,Robert
3,M,Michael
4,M,William


## Pre Procesado

### Limpieza

In [9]:
data = nombres_df

In [10]:
data.head(5)

,Gender,Name
0,M,James
1,M,John
2,M,Robert
3,M,Michael
4,M,William


In [11]:
def clean_tweet(tweet):
    tweet = BeautifulSoup(tweet, "lxml").get_text()
    
    # Eliminamos la @ y su mención
    tweet = re.sub(r"@[A-Za-z0-9]+", ' ', tweet) #https://regexr.com/ --> loquizar cualquier cosa que empiece con @+lo que sea y se sutituye por ' '

    # Eliminamos los links de las URLs
    tweet = re.sub(r"https?://[A-Za-z0-9./]+", ' ', tweet) #el ? significa que el caracter anterior es opcional
    
    # Nos quedamos solamente con los caracteres
    tweet = re.sub(r"[^a-zA-Z.!?']", ' ', tweet) #eliminar todo menos va con el ^ y pongo en []
    
    # Eliminamos espacios en blanco adicionales
    tweet = re.sub(r" +", ' ', tweet)
    
    return tweet

In [12]:
data_clean = [clean_tweet(tweet) for tweet in data.Name]

In [13]:
data.shape

(147269, 2)

In [14]:
set(data.Gender)

{'F', 'M'}

In [15]:
#Para cambiar el F a 0 y M a 1
data_labels = data.Gender.values
data_labels[data_labels == 'F'] = 0
data_labels[data_labels == 'M'] = 1
data_labels

array([1, 1, 1, ..., 1, 1, 1], dtype=object)

In [16]:
data_clean

['James',
 'John',
 'Robert',
 'Michael',
 'William',
 'Mary',
 'David',
 'Joseph',
 'Richard',
 'Charles',
 'Thomas',
 'Christopher',
 'Daniel',
 'Matthew',
 'Elizabeth',
 'Patricia',
 'Jennifer',
 'Anthony',
 'George',
 'Linda',
 'Barbara',
 'Donald',
 'Paul',
 'Mark',
 'Andrew',
 'Steven',
 'Kenneth',
 'Edward',
 'Joshua',
 'Margaret',
 'Brian',
 'Kevin',
 'Jessica',
 'Sarah',
 'Susan',
 'Timothy',
 'Dorothy',
 'Jason',
 'Ronald',
 'Helen',
 'Ryan',
 'Jeffrey',
 'Karen',
 'Nancy',
 'Betty',
 'Lisa',
 'Jacob',
 'Nicholas',
 'Ashley',
 'Eric',
 'Frank',
 'Gary',
 'Anna',
 'Stephen',
 'Jonathan',
 'Sandra',
 'Emily',
 'Amanda',
 'Kimberly',
 'Michelle',
 'Donna',
 'Justin',
 'Laura',
 'Ruth',
 'Carol',
 'Brandon',
 'Larry',
 'Scott',
 'Melissa',
 'Stephanie',
 'Benjamin',
 'Raymond',
 'Samuel',
 'Rebecca',
 'Deborah',
 'Gregory',
 'Sharon',
 'Kathleen',
 'Amy',
 'Cynthia',
 'Alexander',
 'Patrick',
 'Jack',
 'Henry',
 'Angela',
 'Shirley',
 'Emma',
 'Catherine',
 'Katherine',
 'Virgini

### Tokenización

In [17]:
#De una lista de caracteres pasaremos a una lista de números

tokenizer = tfds.deprecated.text.SubwordTextEncoder.build_from_corpus(
    data_clean, target_vocab_size=2**16 #tamaño del vocabulario, en este casos las 65000 más frecuentes. Las menos frecuentes dificilmente la IA las pueda interpretar.
)

data_inputs = [tokenizer.encode(sentence) for sentence in data_clean]

### Padding

In [ ]:
#Padding --> Agrega CEROS hasta igualar todas las frases en su extención

MAX_LEN = max([len(sentence) for sentence in data_inputs]) #max busca la frase más larga
data_inputs = tf.keras.preprocessing.sequence.pad_sequences(data_inputs,
                                                            value=0,
                                                            padding="post",
                                                            maxlen=MAX_LEN)

### Dividimos en los conjuntos de entrenamiento y de testing


In [19]:
#Esto es porque los Tweet estan ordenados 800mil neg y 800mil pos, sacaremos 8000 al azar de la primera mitad y 8000 de la segunda mitad

test_idx = np.random.randint(0, 147269, 10000) #los random de los neg
#test_idx = np.concatenate((test_idx, test_idx+800000)) # los pos se suman al mismo numero de los neg

In [20]:
test_inputs = data_inputs[test_idx] #txt tokenizados y padeado para entrenar la red neuronal
test_labels = data_labels[test_idx] #etiquetas 0 y 1 para entrenar la red neuronal
train_inputs = np.delete(data_inputs, test_idx, axis=0) #lo mismo pero le saco los utilizados arriba
train_labels = np.delete(data_labels, test_idx) #sin axis porque la etiqueta no es una matriz como lo son los txt

In [ ]:
print(test_inputs.shape)
print(test_labels.shape)
print(train_inputs.shape)
print(train_labels.shape)

In [ ]:
print(type(test_inputs))
print(type(test_labels))
print(type(train_inputs))
print(type(train_labels))

In [23]:
test_inputs = np.asarray(test_inputs).astype('float32')
test_labels = np.asarray(test_labels).astype('float32')
train_inputs = np.asarray(train_inputs).astype('float32')
train_labels = np.asarray(train_labels).astype('float32')

# Fase 3: Construción del modelo

In [24]:
#Aquí comienza la CONVULSION, es decir, la aplicación de los filtros

class DCNN(tf.keras.Model): #instanciamos una clase
    
    def __init__(self,
                 vocab_size, #tamaño del vocab, dado por el tokenizador
                 emb_dim=128, #cada palabra será resumida a un numero de 128 caraceteres
                 nb_filters=50, #se pondrán 50 filtros de 4, 3 y 2 palabras
                 FFN_units=512, #número de neuronas de la capa oculta
                 nb_classes=2, #número de categorias de salida, en este caso son 2 por o neg
                 dropout_rate=0.1, #se usa para el entrenamiento, algunas neuronas se apagan para que no aprendan todas juntas, en cada vuelta el 10% se apagará 
                 training=False, #en la fase de predicción las neuronas funcionan todas
                 name="dcnn"): #le damos un nombre al modelo
        super(DCNN, self).__init__(name=name)
        
        self.embedding = layers.Embedding(vocab_size,
                                          emb_dim) #tamaño del vocab
        self.bigram = layers.Conv1D(filters=nb_filters,
                                    kernel_size=2, #filtro de 2 palabras
                                    padding="valid",
                                    activation="relu") #cdo se activa la neurona? con esto que anda bien, es automático
        self.trigram = layers.Conv1D(filters=nb_filters,
                                     kernel_size=3,#filtro de 3 palabras
                                     padding="valid",
                                     activation="relu")
        self.fourgram = layers.Conv1D(filters=nb_filters,
                                      kernel_size=4, #filtro de 4 palabras
                                      padding="valid",
                                      activation="relu")
#El pooling comprimirá la info        
        self.pool = layers.GlobalMaxPool1D() # No tenemos variable de entrenamiento
                                             # así que podemos usar la misma capa 
                                             # para cada paso de pooling

        self.dense_1 = layers.Dense(units=FFN_units, activation="relu")
        self.dropout = layers.Dropout(rate=dropout_rate) #esto es para que no colapse la red
        if nb_classes == 2:
            self.last_dense = layers.Dense(units=1,
                                           activation="sigmoid")#para clasificacion binaria, si hubiera más categorias no podría usarse
        else:
            self.last_dense = layers.Dense(units=nb_classes,
                                           activation="softmax")#para pasar de 2 categorias/binaria a más de dos
    
    def call(self, inputs, training):
        x = self.embedding(inputs)
        x_1 = self.bigram(x)
        x_1 = self.pool(x_1)
        x_2 = self.trigram(x)
        x_2 = self.pool(x_2)
        x_3 = self.fourgram(x)
        x_3 = self.pool(x_3)
        
        merged = tf.concat([x_1, x_2, x_3], axis=-1) # (batch_size, 3 * nb_filters)
        merged = self.dense_1(merged)
        merged = self.dropout(merged, training)
        output = self.last_dense(merged)
        
        return output

# Paso 4: Aplicación

## Configuración

In [25]:
#concretizar todas las variables y parametros que se crearon arriba

VOCAB_SIZE = tokenizer.vocab_size # 65540

EMB_DIM = 200 #cada palabra se identificara con 200 cordenadas
NB_FILTERS = 100 #numero de filtros de la red
FFN_UNITS = 256 #numero de unidades de feet fowards en la capa oculta
NB_CLASSES = 2 #len(set(train_labels)) --> según las etiquetas de entrenamiento

DROPOUT_RATE = 0.2 #20% de las neuronas se apagan en el entrenamiento

BATCH_SIZE = 32 #se probará de 32 en 32 elementos = tweets
NB_EPOCHS = 10 # el data set se iterará 5 veces para mejorar la precisión

## Entrenamiento

In [ ]:
#se creea el OBJETO Deep Convutional Neuronal Network
#se le pasa la configuración anterior

Dcnn = DCNN(vocab_size=VOCAB_SIZE,
            emb_dim=EMB_DIM,
            nb_filters=NB_FILTERS,
            FFN_units=FFN_UNITS,
            nb_classes=NB_CLASSES,
            dropout_rate=DROPOUT_RATE)

In [27]:
#¿qué haremos cdo el proceso de clasificacion es bianario?

if NB_CLASSES == 2:
    Dcnn.compile(loss="binary_crossentropy",
                 optimizer="adam",
                 metrics=["accuracy"])

#si no es binario se resuelve de la siguiente manera:

else:
    Dcnn.compile(loss="sparse_categorical_crossentropy",
                 optimizer="adam",
                 metrics=["sparse_categorical_accuracy"])

In [ ]:
#Esto es para que si se corta la sesión de GoogleColab y no arrancar desde el principio
#SIRVE PARA TENSOR FLOW

checkpoint_path = "/content/drive/Othercomputers/Mi PC/02. Programacion/Proyectos/prediction-of-gender-names/ckpt"

ckpt = tf.train.Checkpoint(Dcnn=Dcnn)

ckpt_manager = tf.train.CheckpointManager(ckpt, checkpoint_path, max_to_keep=5)

if ckpt_manager.latest_checkpoint:
    ckpt.restore(ckpt_manager.latest_checkpoint)
    print("Último checkpoint restaurado!!")

In [ ]:
#Ajustar los datos

Dcnn.fit(train_inputs,
         train_labels,
         batch_size=BATCH_SIZE,
         epochs=NB_EPOCHS)
ckpt_manager.save()

## Evaluación

In [ ]:
#aqui aplica lo entrenado a tweets que no ha leido antes

results = Dcnn.evaluate(test_inputs, test_labels, batch_size=BATCH_SIZE)
print(results)

Aquí se prueba con nombres, con el entrenamiento anterior

In [ ]:
Dcnn(np.array([tokenizer.encode("Wolfgang Amadeus Mozart")]), training=False).numpy()

In [ ]:
tokenizer.encode("nick")